In [ ]:
pip install biopython

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 32.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from Bio import Entrez
from time import sleep

# Set your email — this is required by NCBI
Entrez.email = "your_email@example.com"

def fetch_pubmed_abstracts(query, max_results=100):
    # Search PubMed
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()

    id_list = record["IdList"]
    print(f"Found {len(id_list)} articles for query: '{query}'")

    abstracts = []
    for pmid in id_list:
        try:
            # Fetch article metadata
            fetch_handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="text")
            abstract_text = fetch_handle.read()
            abstracts.append((pmid, abstract_text))
            fetch_handle.close()
            sleep(0.5)  # Be kind to NCBI servers
        except Exception as e:
            print(f"Error fetching PMID {pmid}: {e}")
            continue

    return abstracts

# Run the script
query = "Alzheimer's disease"
abstracts = fetch_pubmed_abstracts(query, max_results=100)

# Save to file
with open("alzheimers_pubmed_abstracts.txt", "w", encoding="utf-8") as f:
    for pmid, abstract in abstracts:
        f.write(f"PMID: {pmid}\n")
        f.write(abstract + "\n\n")


Found 100 articles for query: 'Alzheimer's disease'
